#Attempt at framework for heterogenous systems

In [342]:
class DistCompUnit:
    
    # Consider importing this calculation
    # Note the global value must be more than 0 for the mul. masks to work
    __GLOBAL_ROLES_LIST__ = []
    __PROCESSING_POWER__ = 0
    __MEMORY_CAPABILITIES__ = 0
    __GPU_POWER__ = 0
    __NETWORK_SPEED__ = 0
    __STORAGE__CAP = 0
    __FLEX__ = 0
    __GRADE_VECTOR__ = []
    __ROLE__ = 0
    
    def __init__(self, grade_vector):
        #print('Created Container for Work unit')
        self.__GRADE_VECTOR__ = grade_vector
        self.__PROCESSING_POWER__ = self.__GRADE_VECTOR__[0]
        self.__MEMORY_CAPABILITIES__ = self.__GRADE_VECTOR__[1]
        self.__GPU_POWER__ = self.__GRADE_VECTOR__[2]
        self.__NETWORK_SPEED__ = self.__GRADE_VECTOR__[3]
        self.__STORAGE__CAP = self.__GRADE_VECTOR__[4]
        # TODO: check for vector size
        self.__generate_flexibility__()
        
    def __generate_flexibility__(self):
        """
        A very basic variant of flexibility grading (or the null method).  
        Creates a grade on how flexible the system is to all types of task,
        such that priority is pivoted by flexibility ratings. 
        """
        temp_grade = 0
        #print(self.__GRADE_VECTOR__[0])
        if self.__PROCESSING_POWER__ > 0:
            temp_grade += self.__PROCESSING_POWER__ * 3
        if self.__MEMORY_CAPABILITIES__ > 1:
            temp_grade += self.__MEMORY_CAPABILITIES__ * 2
        if self.__GPU_POWER__ > 1:
            temp_grade += self.__GPU_POWER__ * 2
        if self.__NETWORK_SPEED__ > 1:
            temp_grade += self.__NETWORK_SPEED__
        if self.__STORAGE__CAP > 1:
            temp_grade += self.__STORAGE__CAP 
            
        self.__FLEX__ = temp_grade
        #print(temp_grade)
        
    def get_scalar_grade(self, role_grade_vector):
        
        #TODO: Merge with __generate_flexibility__
        temp_grade = 0
        #print(self.__GRADE_VECTOR__[0])
        if role_grade_vector[0] > 0:
            temp_grade += role_grade_vector[0] * 3
        if role_grade_vector[1] > 1:
            temp_grade += role_grade_vector[1] * 2
        if role_grade_vector[2] > 1:
            temp_grade += role_grade_vector[2] * 2
        if role_grade_vector[3] > 1:
            temp_grade += role_grade_vector[3]
        if role_grade_vector[4] > 1:
            temp_grade += role_grade_vector[4]
            
        return temp_grade
    
    def get_flex(self):
        return self.__FLEX__
        
    def compare_role(self, role_grade_vector):
        """
        Used to compare roles from a 5 element list of values.  
        Returns a binary list if the role is satisfied or not.  
        """
        # Return a binary true if it can handle the role
        role_satisfy = [0, 0, 0, 0, 0]
        if self.__PROCESSING_POWER__ > role_grade_vector[0]:
            role_satisfy[0] = 1
        if self.__MEMORY_CAPABILITIES__ > role_grade_vector[1]:
            role_satisfy[1] = 1
        if self.__GPU_POWER__ > role_grade_vector[2]:
            role_satisfy[2] = 1
        if self.__NETWORK_SPEED__ > role_grade_vector[3]:
            role_satisfy[3] = 1
        if self.__STORAGE__CAP > role_grade_vector[4]:
            role_satisfy[4] = 1
        return role_satisfy
    
    def choose_role(self, role_grade_vector, available_roles):
        """
        Choses the best role from the vector and available roles
        """
        role_determine = None
        # determine the roles you can satisfy
        role_satisfy = self.compare_role(role_grade_vector)
        # print(role_satisfy)
        # find the available roles left, and decide and grade from there.
        relevant_roles = [i*j for i, j in zip(role_satisfy, available_roles)]
        print('relevant roles: ', relevant_roles)
        scalar_grade_vector = [i*j for i, j in zip(relevant_roles, role_grade_vector)]
        print('grade vectors of relevant roles: ', scalar_grade_vector, max(scalar_grade_vector))
        if max(scalar_grade_vector) == 0:
            return role_determine
        choose_list = [i for i,x in enumerate(scalar_grade_vector) if x == max(scalar_grade_vector)]
        print('role (positions) to choose from: ', choose_list)
        if len(choose_list) > 0:
            print('chosen role', choose_list[0])
            role_determine = choose_list[0]
        
        return role_determine
    
    def assign_role(self, global_roles = None, current_available_roles = None):
        #TODO: Role list
        # Choose vector, assign, update vector
        if global_roles is None:
            if self.__GLOBAL_ROLES_LIST__ == []:
                print('empty, no assignment')
            else:
                # TODO: Need to make better behavior for null selection - perhaps
                # create an 'available array?'
                selected_role = self.choose_role(self.__GLOBAL_ROLES_LIST__, [1,1,1,1,1])
                if selected_role is not None:
                    self.__ROLE__ = selected_role
                else:
                    print("no relevant role")
        else:
            if min(global_roles) == 0:
                print('incorrect format')
            else:
                self.__GLOBAL_ROLES_LIST__ = global_roles
                if current_available_roles is not None:
                    selected_role = self.choose_role(self.__GLOBAL_ROLES_LIST__, current_available_roles)
                    if selected_role is not None:
                        self.__ROLE__ = selected_role
                    else:
                        print("no relevant role")
                else:
                    selected_role = self.choose_role(self.__GLOBAL_ROLES_LIST__, [1,1,1,1,1])
                    if selected_role is not None:
                        self.__ROLE__ = selected_role
                    else:
                        print("no relevant role")


In [343]:
grade_vector = [1, 1, 5, 2, 5]
testContainer = DistCompUnit(grade_vector)

In [344]:
testContainer.get_flex()

20

In [345]:
testContainer.compare_role([1,1,4,6,7])

[0, 0, 1, 0, 0]

In [346]:
testContainer.choose_role([1,1,1,1,7], [1,1,1,1,0])

relevant roles:  [0, 0, 1, 1, 0]
grade vectors of relevant roles:  [0, 0, 1, 1, 0] 1
role (positions) to choose from:  [2, 3]
chosen role 2


2

In [347]:
testContainer.assign_role(global_roles=[1,1,1,1,1])

relevant roles:  [0, 0, 1, 1, 1]
grade vectors of relevant roles:  [0, 0, 1, 1, 1] 1
role (positions) to choose from:  [2, 3, 4]
chosen role 2


In [358]:
testContainer.assign_role(global_roles=[1,1,6,1,1], current_available_roles=[1,1,1,1,1])

relevant roles:  [0, 0, 0, 1, 1]
grade vectors of relevant roles:  [0, 0, 0, 1, 1] 1
role (positions) to choose from:  [3, 4]
chosen role 3


In [359]:
testContainer.assign_role()

relevant roles:  [0, 0, 0, 1, 1]
grade vectors of relevant roles:  [0, 0, 0, 1, 1] 1
role (positions) to choose from:  [3, 4]
chosen role 3
